In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim 


In [2]:
torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)

In [28]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [40]:
training_data = [
    ("The dog ate the apple".split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ("Everybody read that book".split(), ["NN", 'V', 'DET', 'NN'])
]

In [41]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [42]:
tag_to_ix = {"DET": 0, "NN":1, "V": 2}

In [43]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [44]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()

        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [45]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [46]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
tensor([[-1.2827, -0.9969, -1.0393],
        [-1.3977, -0.9455, -1.0097],
        [-1.5130, -0.9043, -0.9811],
        [-1.5020, -0.8951, -0.9977],
        [-1.4343, -0.9518, -0.9790]])


In [48]:
for epoch in range(10):
    for sentence, tags in training_data:
        model.zero_grad()

        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [49]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    print(tag_scores)

tensor([[-1.0416, -0.9911, -1.2876],
        [-1.1864, -0.9021, -1.2414],
        [-1.3259, -0.8430, -1.1907],
        [-1.1959, -0.8821, -1.2600],
        [-1.2183, -0.9036, -1.2067]])
